In [3]:
import pandas as pd
import numpy as np
import platform
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
import plotly.express as px

from utils import *

/Applications/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [2]:
# Load data
if platform.system() == 'Windows':
    path = '.\data\stock_port.csv'
else:
    path = './data/stock_port.csv'
df = load_data(path)

# Convert df to sparse matrix
sp_matrix, row_ind_dict, col_ind_dict = convert_data_sparse_matrix(df)

# Basic Info
print('Dimension of sparse_matrix is ', sp_matrix.shape)
row_dim = sp_matrix.shape[0]
col_dim = sp_matrix.shape[1]

# Calculate shareholding % by stock_code
sp_matrix_stock = sp_matrix / np.sum(sp_matrix, axis = 1).reshape(row_dim, -1)

# Calculate shareholding % by shareholder
# sp_matrix_shareholder = sp_matrix / np.sum(sp_matrix, axis = 1).reshape(row_dim, -1)
sp_matrix_shareholder = sp_matrix / np.sum(sp_matrix, axis = 0).reshape(-1, col_dim)

# Element-wise multiply two matrix
sp_matrix_stock_shareholder = sp_matrix_stock * sp_matrix_shareholder

Dimension of sparse_matrix is  (2020, 709)


In [11]:
clf = IsolationForest(n_estimators=150, random_state=0).fit(sp_matrix_stock)
label = clf.predict(sp_matrix_stock)

/Applications/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



In [12]:
# Visualize the result
X_embedded = TSNE(n_components = 2, perplexity = 100, learning_rate = 200).fit_transform(sp_matrix_stock)
df_tsne = pd.DataFrame(X_embedded, columns = ['X1', 'X2'])
df_tsne['stock_code'] = df_tsne.reset_index()['index'].apply(lambda x: {j:i for i,j in row_ind_dict.items()}[x])

In [13]:
df_tsne['label'] = label
fig = px.scatter(df_tsne,'X1','X2', hover_name = 'stock_code', color = 'label')
fig.show()

In [ ]:
# Sklearn documentation
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

# Original Paper
# https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/tkdd11.pdf